In [1]:
import sys, os
sys.path.insert(0, os.path.join('..', 'includes'))

import deep_atlas
from deep_atlas import FILL_THIS_IN
deep_atlas.initialize_environment()
if deep_atlas.environment == 'COLAB':
    %pip install -q python-dotenv==1.0.0

🎉 Running in a Virtual environment


In [2]:
!pipenv install matplotlib==3.8.2 scikit-learn==1.3.2 numpy==1.26.3 torch==2.1.2 torchvision==0.16.2 xgboost==2.0.3 torchinfo==1.8.0

Courtesy Notice:
Pipenv found itself running within a virtual environment,  so it will 
automatically use that environment, instead of  creating its own for any 
project. You can set
PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and 
create  its own instead.
You can set PIPENV_VERBOSITY=-1 to suppress this warning.
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing matplotlib==3.8.2...
✔ Installation Succeeded
Installing scikit-learn==1.3.2...
✔ Installation Succeeded
Installing numpy==1.26.3...
✔ Installation Succeeded
Installing torch==2.1.2...
✔ Installation Succeeded
Installing torchvision==0.16.2...
✔ Installation Succeeded
Installing xgboost==2.0.3...
✔ Installation Succeeded
Installing torchinfo==1.8.0...
✔ Installation Succeeded
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing depende

In [11]:
import os
import numpy as np
import torch
from PIL import Image
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchinfo import summary
from torchvision.models import resnet18, ResNet18_Weights, AlexNet, AlexNet_Weights , MobileNetV2, MobileNet_V2_Weights
import torchvision.models as models

In [4]:
image_paths, labels = [], []

folders = [folder for folder in os.listdir("./images") if os.path.isdir(os.path.join("./images", folder))]

# For each folder, store the image paths and labels
for label, folder in enumerate(folders):
    folder = f"./images/{folder}"
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_paths.append(os.path.join(folder, filename))
            labels.append(
                label
            )  

print(f"Found {len(image_paths)} images")
print(f"Labels: {set(labels)}")

Found 7390 images
Labels: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34}


In [5]:
# First split into train/test
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)
# Then split train into train/val
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_paths, train_labels, test_size=0.25, random_state=42
)

In [18]:

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        image = self.transform(image)
        label = self.labels[idx]
        return image, label

# Create datasets
train_data = ImageDataset(train_paths, train_labels, transform_image)
val_data = ImageDataset(val_paths, val_labels, transform_image)
test_data = ImageDataset(test_paths, test_labels, transform_image)

# Create dataloaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [19]:
transform_image = transforms.Compose(
    [
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],  # Resnet-specific values
        ),
    ]
)


def preprocess_images(image_paths):
    images = []
    for img_path in image_paths:
        # Open the image and read it as Red-Green-Blue values per pixel
        image = Image.open(img_path).convert("RGB")
        # Apply the transform to the image
        image = transform_image(image)
        # Flatten the image into a 1D array of features for XGBoost
        images.append(image.numpy().flatten())
    return np.array(images)


# Convert paths to image data using the preprocess_images function
train_images = preprocess_images(train_paths)
val_images = preprocess_images(val_paths)
test_images = preprocess_images(test_paths)

In [20]:
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [30]:

model_direct = models.resnet18(pretrained=True)
model_direct.fc = nn.Linear(model_direct.fc.in_features, len(set(labels)))  # Adjust final layer for our number of classes
model_direct = model_direct.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_direct.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model_direct.train()
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model_direct(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}')

    # Add after the training loop
model_direct.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_direct(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct / total:.2f}%')

Epoch 1, Loss: 0.1965
Epoch 2, Loss: 0.1368
Epoch 3, Loss: 0.0648
Epoch 4, Loss: 0.0719
Epoch 5, Loss: 0.0515
Epoch 6, Loss: 0.0341
Epoch 7, Loss: 0.0372
Epoch 8, Loss: 0.0214
Epoch 9, Loss: 0.1228
Epoch 10, Loss: 0.0300
Validation Accuracy: 13.13%


In [25]:


def transfer_learn_model(model, num_epochs=10, learning_rate=0.0001):
    """
    Fine-tune a pre-trained model using transfer learning.
    
    Args:
        model: Pre-trained model (e.g., ResNet18)
        num_epochs: Number of epochs to train
        learning_rate: Learning rate for fine-tuning
    """
    # Freeze all layers except the final fully connected layer
    for param in model.parameters():
        param.requires_grad = False
    for param in model.fc.parameters():
        param.requires_grad = True
    
    # Create optimizer with a lower learning rate
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)
    
    # Lists to store metrics
    train_losses = []
    val_losses = []
    val_accuracies = []
    
    best_val_acc = 0.0
    
    print("Starting transfer learning...")
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # Calculate training accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_loss = running_loss/len(train_loader)
        train_acc = 100 * correct / total
        train_losses.append(train_loss)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss = val_loss/len(val_loader)
        accuracy = 100 * correct / total
        
        val_losses.append(val_loss)
        val_accuracies.append(accuracy)
        
        # Update learning rate
        scheduler.step(val_loss)
        
        # Save best model
        if accuracy > best_val_acc:
            best_val_acc = accuracy
            torch.save(model.state_dict(), 'best_transfer_model.pth')
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Training Loss: {train_loss:.4f}, Training Acc: {train_acc:.2f}%')
        print(f'Validation Loss: {val_loss:.4f}, Validation Acc: {accuracy:.2f}%')
        print(f'Learning Rate: {optimizer.param_groups[0]["lr"]:.6f}')
    
    return train_losses, val_losses, val_accuracies






In [26]:
# Create a new model for transfer learning
transfer_model = models.resnet18(pretrained=True)
transfer_model.fc = nn.Linear(transfer_model.fc.in_features, len(set(labels)))  # Adjust final layer
transfer_model = transfer_model.to(device)



/Users/kevincorstorphine/.local/share/virtualenvs/mli-2025-05-05-coursework-rNXVWyyx/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kevincorstorphine/.local/share/virtualenvs/mli-2025-05-05-coursework-rNXVWyyx/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [27]:
# Run transfer learning
train_losses, val_losses, val_accuracies = transfer_learn_model(transfer_model)

Starting transfer learning...
Epoch 1/10:
Training Loss: 1.4086, Training Acc: 4.65%
Validation Loss: 1.3413, Validation Acc: 7.04%
Learning Rate: 0.000100
Epoch 2/10:
Training Loss: 1.1956, Training Acc: 11.82%
Validation Loss: 1.1478, Validation Acc: 15.63%
Learning Rate: 0.000100
Epoch 3/10:
Training Loss: 1.0270, Training Acc: 19.46%
Validation Loss: 0.9874, Validation Acc: 22.46%
Learning Rate: 0.000100
Epoch 4/10:
Training Loss: 0.9000, Training Acc: 25.58%
Validation Loss: 0.8828, Validation Acc: 26.45%
Learning Rate: 0.000100
Epoch 5/10:
Training Loss: 0.7993, Training Acc: 28.89%
Validation Loss: 0.7948, Validation Acc: 29.57%
Learning Rate: 0.000100
Epoch 6/10:
Training Loss: 0.7312, Training Acc: 31.42%
Validation Loss: 0.7318, Validation Acc: 31.39%
Learning Rate: 0.000100
Epoch 7/10:
Training Loss: 0.6687, Training Acc: 32.54%
Validation Loss: 0.6768, Validation Acc: 32.75%
Learning Rate: 0.000100
Epoch 8/10:
Training Loss: 0.6151, Training Acc: 34.37%
Validation Loss: 0.6

In [29]:
# Evaluate on test set
transfer_model.eval()
test_loss = 0.0
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = transfer_model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_loss = test_loss/len(test_loader)
test_acc = 100 * correct / total

print(f'\nTest Results:')
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%')


Test Results:
Test Loss: 0.5712, Test Accuracy: 34.78%
